# PUNTO 2

In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from tqdm import tqdm

In [2]:
class Particle:
    
    def __init__(self, r0, v0, a0, t, m=1, radius=2., Id=0):
        
        self.dt = t[1] - t[0]
        
        
        # Atributos instantaneos
        self.r = r0
        self.v = v0
        self.a = a0
        
        self.m = m
        self.radius = radius
        self.Id = Id
        
        
        self.p = self.m*self.v
        
        self.f = self.m*self.a
        
        # Historial
        
        self.R = np.zeros((len(t),len(r0)))
        self.V = np.zeros_like(self.R)
        self.A = np.zeros_like(self.R)
        
        self.F = np.zeros_like(self.R)
        
        self.P = np.zeros_like(self.R)
    
        # Fisica
        self.K = 20.
        
        self.VEk = np.zeros(len(t))
        
        self.potential_energy = 0  # Inicializar la energía potencial
        
    def Evolution(self,i):
        
        
        self.SetPosition(i)
        self.SetVelocity(i)
        
        self.a = self.f/self.m
        
        # Euler
  #      self.r += self.dt*self.v
  #      self.v += self.dt*self.a
        
        # Euler-Cromer
        self.v += self.dt*self.a
        self.r += self.dt*self.v
        
        
    def CalculateForce(self,p):
        
        d = np.linalg.norm(self.r - p.r)
    
        compresion = self.radius + p.radius - d

        if compresion >= 0:

            Fn = self.K * compresion**3

            self.n = (self.r - p.r) / d
            self.f = np.add(self.f, Fn * self.n)

            # Calcular la energía potencial debido a la compresión
            U = 0.5 * self.K * compresion**2  # Energía potencial elástica

            self.potential_energy += U

     
    def ResetForce(self):
        self.f[:] = 0.
        self.a[:] = 0.
        self.potential_energy = 0
    
    # Setter
    def SetPosition(self,i):
        self.R[i] = self.r
    
    def SetVelocity(self,i):
        self.V[i] = self.v
        self.P[i] = self.m*self.v
        self.VEk[i] = 0.5*self.m*np.dot(self.v,self.v)
    
    # Getter
    def GetPosition(self,scale=1):
        return self.R[::scale]
    
    def GetVelocity(self,scale=1):
        return self.V[::scale]
 
    def GetMomentum(self,scale=1):
        return self.P[::scale]
    
    def GetKineticEnergy(self,scale=1):
        return self.VEk[::scale] 
    
    def CheckLimits(self):
        # Coeficiente de restitución para simular la pérdida de energía en el choque
        e = 0.9  # e=1 significa un rebote perfectamente elástico, e<1 significa que hay alguna pérdida de energía
    
        for i in range(2):
            # Revisar límites en la dirección i (x o y)
            if self.r[i] + self.radius > 20:  # Choque con la pared derecha o superior
                self.v[i] = -self.v[i] * e
                # Calcular energía potencial debido a la deformación de la pared
                compresion = self.r[i] + self.radius - 20
                self.potential_energy += 0.5 * self.K * compresion**2

            elif self.r[i] - self.radius < -20:  # Choque con la pared izquierda o inferior
                self.v[i] = -self.v[i] * e
                # Calcular energía potencial debido a la deformación de la pared
                compresion = -20 - (self.r[i] - self.radius)
                self.potential_energy += 0.5 * self.K * compresion**2

In [3]:
def GetParticles(N, t):
    Particles = []
    
    # Ajustes considerando el radio de las partículas para evitar que se generen en los bordes de la mesa
    pos_min, pos_max = -20 + 2, 20 - 2  # La mesa va de -20 a 20 en ambas direcciones, con un margen para el radio de la partícula
    vel_min, vel_max = -5, 5  # Las velocidades iniciales van de -5 a 5 m/s
    
    for i in range(N):
        # Generar posición inicial aleatoria dentro de los límites de la mesa, ajustando por el radio de la partícula
        r0 = np.random.uniform(low=pos_min, high=pos_max, size=2)
        
        # Generar velocidad inicial aleatoria dentro del rango especificado
        v0 = np.random.uniform(low=vel_min, high=vel_max, size=2)
        
        # Aceleración inicial (puede ser cero si no hay fuerzas externas aplicadas inicialmente)
        a0 = np.array([0., 0.])
        
        # Crear una nueva partícula y agregarla a la lista
        new_particle = Particle(r0, v0, a0, t, m=1, radius=2, Id=i)
        Particles.append(new_particle)
    
    return Particles

In [4]:
dt = 0.001
tmax = 10
N = 10
t = np.arange(0,tmax,dt)
Particles = GetParticles(N,t)

In [5]:
def RunSimulation(t,Particles):
    
    for it in tqdm(range(len(t)), desc='Running simulation', unit=' Steps' ):
        
        for i in range(len(Particles)):
            for j in range(len(Particles)):
                if i!=j:
                    Particles[i].CalculateForce(Particles[j])
        
        for i in range(len(Particles)):
            Particles[i].Evolution(it)
            Particles[i].ResetForce()
            Particles[i].CheckLimits()
            # Aca debes agregar la interaccion con la pared
            
            
    return Particles

In [ ]:
Particles = RunSimulation(t,Particles)

Running simulation:  49%|█████████████████████████▏                         | 4946/10000 [00:02<00:02, 2316.69 Steps/s]

In [ ]:
# Bajamos dimensión de la simulacion
scale = 200
t1 = t[::scale]

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(121)
ax1 = fig.add_subplot(122)

def init():
    
    ax.clear()
    ax.set_xlim(-20,20)
    ax.set_ylim(-20,20)
    
def Update(i):
    
    init()
    ax.set_title(r't =  %.3f s' %(t1[i]))
    
    
    # Queremos calcular la energía total de cinética
    KE = 0. # Kinetic energy
    
    for p in Particles:
        
        x = p.GetPosition(scale)[i,0]
        y = p.GetPosition(scale)[i,1]
        
        vx = p.GetVelocity(scale)[i,0]
        vy = p.GetVelocity(scale)[i,1]
        
        circle = plt.Circle( (x,y), p.radius, color='r', fill=False )
        ax.add_patch(circle)
        
        ax.arrow(x,y,vx,vy,color='k',head_width=0.5,length_includes_head=True)
        
        KE += p.GetKineticEnergy(scale)[i]
        
        ax1.set_title(r'Total kinetic Energy: {:.3f}'.format(KE))
        ax1.scatter(t1[:i], p.GetKineticEnergy(scale)[:i],color='k',marker='.')
        
Animation = anim.FuncAnimation(fig,Update,frames=len(t1),init_func=init)

In [ ]:
# def Plots(Particles, t, scale=1):
#     Writer = anim.writers['ffmpeg']
#     writer_ = Writer(fps=10, metadata=dict(artist='Me'))

In [ ]:
MomentumT = Particles[0].GetMomentum(scale)

In [ ]:
for i in range(1,len(Particles)):
    MomentumT = np.add(MomentumT,Particles[i].GetMomentum(scale))

In [ ]:
fig3 = plt.figure(figsize=(10,5))
ax3 = fig3.add_subplot(221)
ax3.plot(t1,MomentumT[:,0],label='px')
ax3.plot(t1,MomentumT[:,1],label='py')
ax3.legend()

In [ ]:
total_kinetic_energy = []
total_potential_energy = []
total_mechanical_energy = []

# Calcula las energías para cada punto de tiempo
for i in range(len(t1)):  # Asumiendo que t1 es tu arreglo de tiempo reducido
    kinetic_energy = sum(p.GetKineticEnergy(scale)[i] for p in Particles)
    potential_energy = sum(p.potential_energy for p in Particles)  # Asegúrate de que has calculado esto en tu simulación
    mechanical_energy = kinetic_energy + potential_energy
    
    total_kinetic_energy.append(kinetic_energy)
    total_potential_energy.append(potential_energy)
    total_mechanical_energy.append(mechanical_energy)

In [ ]:
# Crear la figura y el eje
fig4, ax4 = plt.subplots(figsize=(10, 6))

# Graficar cada energía contra el tiempo
ax4.plot(t1, total_kinetic_energy, label='Energía Cinética Total', color='blue')
ax4.plot(t1, total_potential_energy, label='Energía Potencial Total', color='green')
ax4.plot(t1, total_mechanical_energy, label='Energía Mecánica Total', color='red')

# Añadir títulos y etiquetas
ax4.set_title('Energías Totales vs. Tiempo')
ax4.set_xlabel('Tiempo (s)')
ax4.set_ylabel('Energía (Joules)')

# Mostrar leyenda
ax4.legend()

# Mostrar el gráfico
plt.show()

In [ ]:
# Inicializar una lista para almacenar el momento angular total en cada paso de tiempo
total_angular_momentum_z = []

for i in range(len(t)):  # Asumiendo que 't' es tu array de tiempo
    L_z_total = 0  # Momento angular total en el eje z para el paso de tiempo actual
    
    for p in Particles:
        # Obtener posición y momento lineal (m*v) de la partícula
        r = p.GetPosition()[i]  # Asume GetPosition devuelve un array [x, y]
        p = p.GetMomentum()[i]  # Asume GetMomentum devuelve un array [m*v_x, m*v_y]
        
        # Calcular el momento angular de esta partícula en z
        L_z = r[0]*p[1] - r[1]*p[0]
        
        # Sumar al total
        L_z_total += L_z
    
    # Añadir el momento angular total de este paso de tiempo a la lista
    total_angular_momentum_z.append(L_z_total)

In [ ]:
# Crear la figura y el eje
fig, ax5 = plt.subplots(figsize=(10, 6))

# Graficar el momento angular total en el eje z en función del tiempo
ax5.plot(t, total_angular_momentum_z, label='Momento Angular Total en Z', color='purple')

# Añadir título y etiquetas
ax5.set_title('Momento Angular Total en Z vs. Tiempo')
ax5.set_xlabel('Tiempo (s)')
ax5.set_ylabel('Momento Angular (kg*m^2/s)')

# Mostrar leyenda
ax5.legend()

# Mostrar el gráfico
plt.show()